In [7]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [8]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    nx, ny, nz = cells.shape
    for x in range(nx - 1):
        for y in range(ny - 1):
            for z in range(nz - 1):
                case_num = getCaseNum(x, y, z, thres, cells)
                idx = []
                for case in case_num:
                    if case == -1:
                        break
                    edge_offset = CaseNum2EdgeOffset[case]
                    x1 = x + edge_offset[0]
                    y1 = y + edge_offset[1]
                    z1 = z + edge_offset[2]
                    x2 = x + edge_offset[3]
                    y2 = y + edge_offset[4]
                    z2 = z + edge_offset[5]
                    vertex1 = np.array([x1, y1, z1])
                    vertex2 = np.array([x2, y2, z2])
                    cell1 = cells[x1, y1, z1]
                    cell2 = cells[x2, y2, z2]
                    
                    # avoid division by zero
                    if abs(cell2 - cell1) < 1e-6:
                        t = 0.5
                    else:
                        t = (thres - cell1) / (cell2 - cell1)
                    ip = vertex1 + t * (vertex2 - vertex1)
                    key = tuple(np.round(ip, 5))
                    if key not in vertex_array:
                        vertex_array[key] = len(vertex_array)
                    idx.append(vertex_array[key])
                    
                for i in range(0, len(idx), 3):
                    if i + 2 < len(idx):
                        face_array.append([idx[i], idx[i+1], idx[i+2]])
                    
                
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertices = [None] * len(vertex_array)
    for key, index in vertex_array.items():
        vertices[index] = np.array(key)
    
    return np.array(vertices), np.array(face_array)

In [9]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
0.8596706390380859 s

Time taken by algorithm
----------------------------------------
0.9348781108856201 s
